## Import libraries

In [ ]:
from postprocessing import *

In [2]:
%cd mmaction2

D:\jupyterNotebook\C8099\Project\mmaction2


## Video Inference and Visualisation

In [20]:
# Paths to configuration and checkpoint files
config_path_task_normal = 'configs/recognition/swin/swin-base-goosebumps-project-task-normal.py'
checkpoint_path_task_normal = 'work_dirs/trained-model/task-normal-model.pth'

config_path_task_subclass = 'configs/recognition/swin/swin-base-goosebumps-project-task-subclass.py'
checkpoint_path_task_subclass = 'work_dirs/trained-model/task-subclass-model.pth'

input_folder = 'data/Goosebumps_videos_multi_task'
output_folder = 'inference_output_videos'

# Folders to exclude
exclude_folders = {'class_2_augment', 'class_3_augment'}

# Create Output Folder
os.makedirs(output_folder, exist_ok=True)

# Initialise the model
model_task_normal = init_recognizer(config_path_task_normal, checkpoint_path_task_normal, device="cuda:0")
model_task_subclass = init_recognizer(config_path_task_subclass, checkpoint_path_task_subclass, device="cuda:0")

# Label mapping
label_map_task_normal = {0: 'No Goosebumps', 1: 'Goosebumps'}
label_map_task_subclass = {0: 'Small bump', 1: 'Medium bump', 2: 'Large bump'}

Loads checkpoint by local backend from path: work_dirs/project-experiment-result/swin-base/task-normal/best_acc_top1_epoch_28.pth
Loads checkpoint by local backend from path: work_dirs/project-experiment-result/swin-base/task-subclass/epoch_36.pth


In [23]:
# Iterate through each folder in the input directory
for view_folder in os.listdir(input_folder):
    # Construct the path to the current view folder
    view_path = os.path.join(input_folder, view_folder)

    # Skip if the path is not a directory
    if not os.path.isdir(view_path):
        continue

    # Collect video metadata from the current view folder
    videos_info = collect_videos_info(view_path, exclude_folders)

    # Process and combine videos, then save the output
    process_videos(videos_info, view_path, output_folder, model_task_normal, 
                   model_task_subclass, label_map_task_normal, label_map_task_subclass)

print(f'Successfully output all the videos')

Output and saved video to: inference_output_videos\101_Dom_arm_combined.mp4
Output and saved video to: inference_output_videos\103_Dom_arm_combined.mp4
Output and saved video to: inference_output_videos\105_Dom_arm_combined.mp4
Output and saved video to: inference_output_videos\107_Dom_arm_combined.mp4
Output and saved video to: inference_output_videos\108_Dom_arm_combined.mp4
Output and saved video to: inference_output_videos\109_Dom_arm_combined.mp4
Output and saved video to: inference_output_videos\111_Dom_arm_combined.mp4
Output and saved video to: inference_output_videos\115_Dom_arm_combined.mp4
Output and saved video to: inference_output_videos\117_Dom_arm_combined.mp4
Output and saved video to: inference_output_videos\121_Dom_arm_combined.mp4
Output and saved video to: inference_output_videos\123_Dom_arm_combined.mp4
Output and saved video to: inference_output_videos\124_Dom_arm_combined.mp4
Output and saved video to: inference_output_videos\127_Dom_arm_combined.mp4
Output and s

In [24]:
# Define input and output folders
input_folder = 'inference_output_videos'
output_folder = 'inference_result_visualization_videos'

# Create output directory if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Retrieve all unique sequence numbers from video filenames
sequence_numbers = set()
for filename in os.listdir(input_folder):
    if filename.endswith('_combined.mp4'):
        seq_number = filename.split('_')[0]
        sequence_numbers.add(seq_number)
sequence_numbers = sorted(sequence_numbers)

# Process each sequence
for seq in sequence_numbers:
    print(f'Processing sequence {seq}')

    # Construct file paths for the four video categories
    right_thigh_path = os.path.join(input_folder, f'{seq}_Right_thigh_combined.mp4')
    left_thigh_path = os.path.join(input_folder, f'{seq}_Left_thigh_combined.mp4')
    dom_arm_path = os.path.join(input_folder, f'{seq}_Dom_arm_combined.mp4')
    dom_calf_path = os.path.join(input_folder, f'{seq}_Dom_calf_combined.mp4')

    # Open video files
    cap_right_thigh = cv2.VideoCapture(right_thigh_path)
    cap_left_thigh = cv2.VideoCapture(left_thigh_path)
    cap_dom_arm = cv2.VideoCapture(dom_arm_path)
    cap_dom_calf = cv2.VideoCapture(dom_calf_path)

    # Retrieve video properties
    fps = int(cap_right_thigh.get(cv2.CAP_PROP_FPS))
    width = int(cap_right_thigh.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap_right_thigh.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Get frame counts for each video
    frame_counts = {
        'right_thigh': int(cap_right_thigh.get(cv2.CAP_PROP_FRAME_COUNT)),
        'left_thigh': int(cap_left_thigh.get(cv2.CAP_PROP_FRAME_COUNT)),
        'dom_arm': int(cap_dom_arm.get(cv2.CAP_PROP_FRAME_COUNT)),
         'dom_calf': int(cap_dom_calf.get(cv2.CAP_PROP_FRAME_COUNT))
    }

    # Determine the minimum frame count among the videos
    min_frame_count = min(frame_counts.values())
    
    # Set up the output video properties
    output_width = 1920
    output_height = 1080
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(os.path.join(output_folder, f'{seq}_inference_result.mp4'), fourcc, fps, (output_width, output_height))

    # Process each frame up to the minimum frame count
    for _ in range(min_frame_count):
        ret_right_thigh, frame_right_thigh = cap_right_thigh.read()
        ret_left_thigh, frame_left_thigh = cap_left_thigh.read()
        ret_dom_arm, frame_dom_arm = cap_dom_arm.read()
        ret_dom_calf, frame_dom_calf = cap_dom_calf.read()

        # Check if frames were successfully read
        if not (ret_right_thigh and ret_left_thigh and ret_dom_arm and ret_dom_calf):
            break

        # Create a black background for merging frames
        merged_frame = np.zeros((output_height, output_width, 3), dtype=np.uint8)

        # Stitch video frames to the appropriate position
        merged_frame[0:540, 0:734] = frame_right_thigh  # Top-left
        merged_frame[0:540, 734:1468] = frame_dom_arm  # Top-right
        merged_frame[540:1080, 0:734] = frame_left_thigh  # Bottom-left
        merged_frame[540:1080, 734:1468] = frame_dom_calf  # Bottom-right

        # Write the merged frame to the output video file
        out.write(merged_frame)
        print(f'Finished processing sequence {seq}')

    # Release resources for each video capture and the video writer
    cap_right_thigh.release()
    cap_left_thigh.release()
    cap_dom_arm.release()
    cap_dom_calf.release()
    out.release()

print("Processing complete.")

Processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing sequence 101
Finished processing